In [64]:
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import animation
import pickle
from itertools import combinations
from scipy import stats

Read in the decoding score data

In [65]:
#read in decoding accuracy scores

#set up directory to read in decoding scores
S = 'S03'
rootdir = os.getcwd()
outdir = rootdir + '/plots'
in_filepath = rootdir + '/' + S + '_IMbased_allscores.pkl'

#load scores from the pickle file into all_scores, a dictionary of the following format: 
# key: tuple of two trial codes representing a pair of images ex. (1,2)
# value: a 1x1081 list of decoding scores at each timepoint for those two images
in_file = open(in_filepath,'rb')
all_scores = pickle.load(in_file)
in_file.close()

#all_scores has 7750 items, representing every pair of images 1-125
print(len(all_scores))

7750


Build the MEG RDM

In [66]:
#convert the all_scores dictionary to a 3D numpy array (MEG RDM)
nrows = 125
nsamples = len(all_scores[1,2])

#initialize a numpy array of zeros, size: 125x125x1081
rdm_meg = np.zeros((nrows,nrows,nsamples))

#iterate through each entry in all_scores
for key,value in all_scores.items():

    #each key dictates the row and column number we want
    row = key[0] - 1
    col = key[1] - 1

    #fill in the corresponding list of decoding accuracies in symmetric positions in the array
    rdm_meg[row,col] = value
    rdm_meg[col,row] = value




plot the MEG RDM 

In [86]:
#plot a single snapshot of the MEG RDM -  see figure in plots folder
plt.imshow(rdm_meg[:,:,540],cmap='Blues')
plt.title('MEG RDM at ~350 ms')
plt.colorbar()

#Use animation feature to visualize RDM over time
#animation currently lags, need to plot more efficiently and add label for timepoint
fig = plt.figure()

def animate(i):
    plt.imshow(rdm_meg[:,:,i],cmap='Blues')

anim = animation.FuncAnimation(fig,animate,interval=5)

Build Model RDMs based on trial structure

In [82]:
#these three helper functions generate trial codes for the (5 actors x 5 expr x 5 views)
#input: number 1-5 representing actor, expression or view code
#output: list of all trial codes correspoding to that actor, expression or view

#these functions are used as arguments (trialcode_func) for later functions build_trial_key and build_model_rdm

def get_trials_by_expression(exp_code):
    trial_codes = []
    start_code = (exp_code*5) - 4
    
    for i in range(start_code,125,25):
        for j in range(i,i+5):
            trial_codes.append(j)
    return trial_codes

def get_trials_by_id(actor_code):
    start_code = (actor_code * 25) - 24
    return [i for i in range(start_code,start_code+25)]

def get_trials_by_view(view_code):
    return [i for i in range(view_code, 126,5)]

print('Trial codes for expression 1: ', get_trials_by_expression(1))

#helper function for building model rdms
def build_trial_key(trialcode_func):
    #input: one of the three trial code functions above
    #output: a dict mapping each trial code (1-125) to its corresponding id, expression, or view code (1-5)
    key = {}
    for i in range(5):
        codes = (trialcode_func(i+1))
        #print(codes)
        
        for code in codes:
            key[code] = i+1

    return key

print('Expression codes for every trial code: ', build_trial_key(get_trials_by_expression))

Trial codes for expression 1:  [1, 2, 3, 4, 5, 26, 27, 28, 29, 30, 51, 52, 53, 54, 55, 76, 77, 78, 79, 80, 101, 102, 103, 104, 105]
Expression codes for every trial code:  {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 31: 2, 32: 2, 33: 2, 34: 2, 35: 2, 56: 2, 57: 2, 58: 2, 59: 2, 60: 2, 81: 2, 82: 2, 83: 2, 84: 2, 85: 2, 106: 2, 107: 2, 108: 2, 109: 2, 110: 2, 11: 3, 12: 3, 13: 3, 14: 3, 15: 3, 36: 3, 37: 3, 38: 3, 39: 3, 40: 3, 61: 3, 62: 3, 63: 3, 64: 3, 65: 3, 86: 3, 87: 3, 88: 3, 89: 3, 90: 3, 111: 3, 112: 3, 113: 3, 114: 3, 115: 3, 16: 4, 17: 4, 18: 4, 19: 4, 20: 4, 41: 4, 42: 4, 43: 4, 44: 4, 45: 4, 66: 4, 67: 4, 68: 4, 69: 4, 70: 4, 91: 4, 92: 4, 93: 4, 94: 4, 95: 4, 116: 4, 117: 4, 118: 4, 119: 4, 120: 4, 21: 5, 22: 5, 23: 5, 24: 5, 25: 5, 46: 5, 47: 5, 48: 5, 49: 5, 50: 5, 71: 5, 72: 5, 73: 5, 74: 5, 75: 5, 96: 5, 97: 

In [73]:
#build a model rdm for each condition

def build_model_rdm(trialcode_func):

    #build the key linking trial codes to exp/id/view codes
    key = build_trial_key(trialcode_func)

    n = len(key)

    #initialize a numpy array filled with ones, size 125x125
    rdm = np.ones((n,n))

    #iterate through each row and column number to build the model rdm
    for r in range(n):

        for c in range(n):

            #if the two trial codes map to the same expression, id or view, fill this entry with 0
            if key[r+1] == key[c+1]:
                rdm[r,c] = 0
                
            #if different, entry remains as 1

    return rdm

rdm_view = build_model_rdm(get_trials_by_view)
rdm_id = build_model_rdm(get_trials_by_id)
rdm_exp = build_model_rdm(get_trials_by_expression)


In [74]:
#plot the 3 model RDM as subplots on a single figure
fig,ax = plt.subplots(1,3)

ax[0].imshow(rdm_id)
ax[0].set_title('Identity')
ax[1].imshow(rdm_exp)
ax[1].set_title('Expression')
ax[2].imshow(rdm_view)
ax[2].set_title('Viewpoint')

#adjust the spacing between subplots to avoid overlap
fig.tight_layout()


#see this figure in plots folder

Compute similarity

In [77]:
#function to correlate one model rdm with every timepoint of the meg rdm
#return a 1x1081 list containing a timecourse of Spearman correlations
def rsa(model, meg):

    timecourse = []

    #extract the upper triangle from the model rdm and flatten it
    model_flat = np.ndarray.flatten(np.triu(model,k=1))

    #iterate through each timepoint
    for i in range(len(meg[0,0])): # 

        #extract the meg rdm for that timepoint
        slice = meg[:,:,i]

        #extract the upper triangle and flatten the meg rdm
        slice_flat = np.ndarray.flatten(np.triu(slice,k=1))

        #compute Spearman's R between the model and meg 
        #not keeping track of p right now but should be changed
        correlation,p = stats.spearmanr(model_flat,slice_flat)

        timecourse.append(correlation)

    return timecourse


rsa_view = rsa(rdm_view,rdm_meg)
rsa_id = rsa(rdm_id,rdm_meg)
rsa_exp = rsa(rdm_exp,rdm_meg)
print(len(rsa_id))



1081


In [78]:
#plot results of rsa
#set ranges for x and y axes
xmin = -0.1
xmax = 0.8
ymin = 0.75
ymax = 0.85
axes = [xmin,xmax,ymin,ymax]

#generate evenly spaced values for the x-axis
nsamples = len(rsa_view)
xvals = np.linspace(xmin,xmax,nsamples)


plt.plot(xvals,rsa_exp,label='expression')
plt.plot(xvals,rsa_id,label='identity')
plt.plot(xvals,rsa_view,label='viewpoint')
plt.axis(axes)
plt.title('Timecourse of Spearman\'s correlations from RSA')
plt.xlabel('time (s)')
plt.ylabel('Spearman\'s R')
plt.legend()